In [ ]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook, tqdm_pandas
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

In [ ]:
!pip install SPARQLWrapper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
target_dir = '/content/drive/My Drive/Colab Notebooks/dataset/Round1/targets'
gt_dir = '/content/drive/My Drive/Colab Notebooks/dataset/Round1/targets/gt'
tables_dir = '/content/drive/My Drive/Colab Notebooks/dataset/Round1/tables'

In [ ]:
df_gt = pd.read_csv(os.path.join(gt_dir, "CTA_Round1_gt.csv"), 
                        header=None,
                        names=['table_id', 'column_id', 'type'],
                        dtype={'column_id': np.int8})

In [ ]:
df_targets = pd.read_csv(os.path.join(target_dir, "CTA_Round1_Targets.csv"), 
                        header=None, 
                        nrows=120,
                        names=['table_id', 'column_id'],
                        dtype={'column_id': np.int8})

In [ ]:
def preporcess_item(word):
    # LAKE
    word = re.sub(' \*? ?(A|a)lso.*', '', word)
    word = re.sub('(\(|\[).*(\)|\])', '', word)
    
    word = re.sub('[^A-Za-z0-9 \-\d+/\d+\?]+', '', word)
    word = re.sub('( |\-){1,}', '_', word)
    word = word.replace("__", "_")
    
    word = re.sub('(^_|_$)', '', word)
    
    return word

In [ ]:
column_items = []
def get_column_items(row):
    global column_items
    table_id = row["table_id"]
    column_id = row["column_id"]
    
    df = pd.read_csv(os.path.join(tables_dir, table_id + ".csv"))
    cells = []
    column = df.iloc[ : , column_id]
    for i, value in column.items():
        value = preporcess_item(str(value))
        if not (pd.isna(value)): 
            cells.append(value)
    column_items.append(cells)

In [ ]:
df_targets.apply(get_column_items, axis=1)

In [ ]:
def get_ontology_classes(item):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
#     sparql.setQuery("select distinct ?class where { dbr:"+item+" a ?class.}")
    sparql.setQuery("""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?type
        WHERE { <http://dbpedia.org/resource/"""+item+"""> rdf:type ?type }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    return [result["type"]["value"] for result in results["results"]["bindings"] 
            if 'http://dbpedia.org/ontology' in result["type"]["value"]]

In [ ]:
items_classes = []

for column in tqdm_notebook(column_items):
    item_classes = []
    for item in tqdm_notebook(column):
        item_classes.append(get_ontology_classes(item))
    items_classes.append(item_classes)

In [ ]:
from collections import Counter

In [ ]:
items_classes_counter = []

In [ ]:
items_classes[3]

In [ ]:
items_classes_counter = []
for column in items_classes:
  temp = []
  for classes in column:
    if classes:
      temp.append(classes[0])
  items_classes_counter.append(Counter(temp))

In [ ]:
items_classes_counter

In [ ]:
annotations = []
for annotation in items_classes_counter:
  if annotation:
    annotations.append(annotation.most_common(1)[0][0])
  else:
    annotations.append('')

In [ ]:
df_annotations = pd.DataFrame(annotations, columns =['db_classes'])

In [ ]:
df_targets
df_targets['db_classes'] = df_annotations['db_classes'].to_numpy()

In [ ]:
df_targets["column_id"] = df_targets.column_id.astype(str)
df_targets["db_classes"] = df_targets.db_classes.astype(str)

In [ ]:
df_targets

In [ ]:
df_targets.to_csv('df_annotations.csv', index=False, header=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)